# Pixelate map
This notebook mainly draw a pixelated map by formatting a json file

In [1]:
import json
import os

import netCDF4 as nc
import numpy as np
import pandas as pd

## Get some data for the map
Before we draw the map, we need to have some data. Here I just copy-paste codes to count max consecutive MPID, calculate an EV score, and calculate maximum daily range loss  
Adding more data is trivia if you follow the same paradigm of formatting the data  

Define some constants

In [2]:
DATA_FILE_DIR = "./data/nasa/"

START_YEAR, END_YEAR = 2010, 2020

NUM_OF_YEARS = END_YEAR - START_YEAR

NUM_OF_MONTHS = 12

NUM_OF_DAYS = {1: 31, 2: 28, 3: 31, 4: 30, 5: 31, 6: 30, 7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12: 31}

The source data format is `netCDF4`.  
First, We need to use any of the source files to extract latitudes and longitudes

In [3]:
file = nc.Dataset(DATA_FILE_DIR+'20110101.nc4')

lat = file.variables['lat'][:].filled()
lon = file.variables['lon'][:].filled()
LON = len(lon)
LAT = len(lat)

# we will use this mask later
mask = file.variables['AvgSurfT_tavg'][0].mask

# remember to close opened files after use
file.close()

In [4]:
def get_tmp(filepath):
    """
    This function extracts temperature data from the given filepath
    
    # Arguements:
        filepath: A string that specifies the file to be read
    # Returns:
        The data temperature in the file
    """
    assert os.path.isfile(filepath), '{} does not exist!'.format(filepath)
    
    file = nc.Dataset(filepath)
    temperature = file.variables['AvgSurfT_tavg'][0]
    file.close()
    
    return temperature.filled(273.15)

Data 1: maximum consecutive MPID (must plug-in day)  
Algorithm: Keep two counters. One records the current consec MPID, and the other records the max consec MPID it has seen so far. After counting one day's MPID, compare the two and keep the larger one. Use `np.where` to adapt this algo to count on arrays  
  
Data 2: average maximum and daily range loss

In [5]:
%%time
# read the scale factors
scale_factors = pd.read_csv("./data/fitted_factors.csv")
percent_loss = scale_factors["Range Loss"].to_numpy()

# max_MPID will record the max MPID of all places in a year
max_MPID = np.ndarray(shape=(LAT, LON))
# curr_MPID will count the consecutive MPID we have seen so far (the counter)
curr_MPID = np.ndarray(shape=(LAT, LON))
# each_year_avg_loss will record each year's avgerage daily range loss
each_year_avg_loss = np.zeros(shape=(NUM_OF_YEARS, LAT, LON))
# max_loss will record the maximum daily range loss
max_loss = np.zeros(shape=(LAT, LON))

for year in range(START_YEAR, END_YEAR):
    print(year, end=' ')
    
    # keep track of the number of days
    i = 0
    # yearly_loss will record each day's range loss of this year
    yearly_loss = np.zeros(shape=(365, LAT, LON))
    
    for month in range(1, NUM_OF_MONTHS+1):
        for day in range(1, NUM_OF_DAYS[month]+1):
            date = "{}{:02d}{:02d}".format(year, month, day)
            filepath = DATA_FILE_DIR + date + '.nc4'
            date_temp = get_tmp(filepath)
            
            # if this place has MPID on this day (temp<253.15K), then curr_MPID+1
            # else, this place has no MPID on this day, which means not consecutive, so we reset the counter to 0;
            curr_MPID = np.where(date_temp<253.15, curr_MPID+1, 0) # 253.15 K = -20 oC
            # this is equivalent to A = max(A, B)
            max_MPID = np.where(curr_MPID>max_MPID, curr_MPID, max_MPID)
            
            # get the range loss of each day in this year
            date_temp = np.round(date_temp-273.15, decimals=1) # convert to oC
            # use the temperature difference as index. e.g. if temperature is -12.5 oC, then its range loss will
            # be the (-12.5+100)*10=875th element in the percent_loss array
            # "+100" means "-(-100)", "*10" means "/0.1"
            index = (date_temp+100)*10
            yearly_loss[i] = percent_loss[index.astype(int)]
            
            i += 1
    
    # calculate the yearly average daily range loss
    each_year_avg_loss[year-START_YEAR] = yearly_loss.mean(axis=0)
    # calculate the yearly maximum daily range loss
    # NOTE: range loss is a negative value, so we use min()
    yearly_max_loss = yearly_loss.min(axis=0)
    max_loss = np.where(yearly_max_loss<max_loss, yearly_max_loss, max_loss)

print('Finished!\n')

2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 Finished!

Wall time: 5min 6s


Data 3: EV score  
EV score is simply converted from daily range loss

In [6]:
# get the daily average range loss
avg_loss = each_year_avg_loss.mean(axis=0)

# calculate score and round it to 1 decimal
avg_score = (avg_loss - avg_loss.min()) / (avg_loss.max() - avg_loss.min()) * 100
avg_score = np.round(avg_score, decimals=1)

## Build the map with pixels

In [7]:
def get_zone(score):
    """
    A simple function to return the corresponding zone of the given score
    """
    assert 0 <= score <= 100, "Score {} is out of valid range [0, 100]".format(score)
    
    if score < 10:
        return "0-10"
    elif score < 20:
        return "10-20"
    elif score < 30:
        return "20-30"
    elif score < 40:
        return "30-40"
    elif score < 50:
        return "40-50"
    elif score < 60:
        return "50-60"
    elif score < 70:
        return "60-70"
    elif score < 80:
        return "70-80"
    elif score < 90:
        return "80-90"
    else:
        return "90-100"

Format a JSON file  
Each pixel is a rectangle shape (Polygon)

In [8]:
%%time
json_file = {'features':[], 'type': 'FeatureCollection'}
for i in range(LAT):
    for j in range(LON):
        if (not mask[i, j]):
            feature = {'geometry':{'coordinates':[], 'type': 'Polygon'},
                       'properties':{},
                       'type': 'Feature'}
            
            # the shape boundary of one pixel. It is represented in a list
            pixel = [[[lon[j]-.125,lat[i]-.125],
                      [lon[j]-.125,lat[i]+.125],
                      [lon[j]+.125,lat[i]+.125],
                      [lon[j]+.125,lat[i]-.125],
                      [lon[j]-.125,lat[i]-.125]]]
            feature['geometry']['coordinates'] = pixel
            
            # if you want to add more information/values just follow this template:
            # feature['properties']['YOUR FEATURE NAME'] = YOUR VALUE
            feature['properties']['EV Zone'] = get_zone(avg_score[i,j])
            feature['properties']['Score'] = avg_score[i,j]
            feature['properties']['Max consecutive MPID'] = int(max_MPID[i,j])
            feature['properties']['Max daily range loss'] = max_loss[i,j]
            
            # append this pixel to the feature list
            json_file['features'].append(feature)

Wall time: 11.1 s


save the map in json format

In [9]:
os.makedirs("./geojson_files", exist_ok=True)
with open('./geojson_files/pixel_data_1.json', 'w') as outfile:
    json.dump(json_file, outfile)